In [1]:
import jpype
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java

In [2]:
from zipfile import *
import zipfile

In [3]:
def read_jar(jar_file):
    zf = zipfile.ZipFile(jar_file, 'r')
    try:
        lst = zf.infolist()
        for zi in lst:
            fn = zi.filename
            print(fn)     
    finally:
        zf.close()

In [4]:
read_jar("zemberek-full.jar")

META-INF/MANIFEST.MF
META-INF/
META-INF/maven/
META-INF/maven/zemberek-nlp/
META-INF/maven/zemberek-nlp/zemberek-all/
META-INF/maven/zemberek-nlp/zemberek-all/pom.xml
META-INF/maven/zemberek-nlp/zemberek-all/pom.properties
zemberek/
zemberek/core/
zemberek/core/StringFloat.class
zemberek/core/embeddings/
zemberek/core/embeddings/WordVectorsTrainer.class
zemberek/core/embeddings/Model$HierarchicalSoftmax.class
zemberek/core/hash/
zemberek/core/hash/LargeNgramMphf.class
zemberek/core/io/
zemberek/core/io/SimpleTextWriter$Builder.class
zemberek/core/io/StringFilters$NullOrEmptyFilter.class
zemberek/core/io/Filter.class
zemberek/core/turkish/
zemberek/core/turkish/TurkishAlphabet.class
zemberek/core/turkish/SecondaryPos.class
zemberek/core/turkish/RootAttribute.class
zemberek/core/concurrency/
zemberek/core/concurrency/BlockingExecutor.class
zemberek/core/ScoredItem.class
zemberek/core/syllable/
zemberek/core/syllable/accepted-syllable-prefixes
zemberek/core/text/
zemberek/core/text/specia

org/antlr/v4/runtime/tree/TerminalNode.class
org/antlr/v4/runtime/tree/xpath/XPathLexerErrorListener.class
org/antlr/v4/runtime/tree/xpath/XPathRuleAnywhereElement.class
META-INF/maven/org.antlr/antlr4-runtime/pom.properties
org/antlr/v4/runtime/atn/AmbiguityInfo.class
org/antlr/v4/runtime/atn/LexerChannelAction.class
org/antlr/v4/runtime/atn/LexerCustomAction.class
org/antlr/v4/runtime/atn/LexerIndexedCustomAction.class
org/antlr/v4/runtime/atn/LexerModeAction.class
org/antlr/v4/runtime/atn/LexerMoreAction.class
org/antlr/v4/runtime/atn/LexerPushModeAction.class
org/antlr/v4/runtime/atn/LexerTypeAction.class
org/antlr/v4/runtime/atn/LL1Analyzer.class
org/antlr/v4/runtime/atn/NotSetTransition.class
org/antlr/v4/runtime/atn/OrderedATNConfigSet.class
org/antlr/v4/runtime/atn/ParseInfo.class
org/antlr/v4/runtime/atn/PlusBlockStartState.class
org/antlr/v4/runtime/atn/PlusLoopbackState.class
org/antlr/v4/runtime/atn/PrecedencePredicateTransition.class
org/antlr/v4/runtime/atn/PredicateEvalI

_zem/io/grpc/netty/shaded/io/netty/channel/CombinedChannelDuplexHandler.class
_zem/io/grpc/netty/shaded/io/netty/channel/SimpleChannelInboundHandler.class
_zem/io/grpc/netty/shaded/io/netty/channel/AbstractChannelHandlerContext$2.class
_zem/io/grpc/netty/shaded/io/netty/channel/ChannelHandlerMask$1.class
_zem/io/grpc/netty/shaded/io/netty/channel/DefaultAddressedEnvelope.class
_zem/io/grpc/netty/shaded/io/netty/channel/AbstractChannel$AbstractUnsafe$2.class
_zem/io/grpc/netty/shaded/io/netty/channel/AbstractChannelHandlerContext$6.class
_zem/io/grpc/netty/shaded/io/netty/channel/ChannelFutureListener$1.class
_zem/io/grpc/netty/shaded/io/netty/channel/ChannelInitializer$1.class
_zem/io/grpc/netty/shaded/io/netty/channel/ChannelPromise.class
_zem/io/grpc/netty/shaded/io/netty/channel/MessageSizeEstimator.class
_zem/io/grpc/netty/shaded/io/netty/channel/AbstractChannelHandlerContext.class
_zem/io/grpc/netty/shaded/io/netty/channel/AbstractServerChannel.class
_zem/io/grpc/netty/shaded/io/n

_zem/io/opencensus/common/Function.class
_zem/io/opencensus/common/Duration.class
_zem/io/opencensus/common/ServerStatsDeserializationException.class
_zem/io/opencensus/common/ServerStatsFieldEnums.class
_zem/io/opencensus/common/Functions$3.class
_zem/io/opencensus/common/TimeUtils.class
_zem/io/opencensus/common/Functions$2.class
_zem/io/opencensus/common/ToDoubleFunction.class
_zem/io/opencensus/common/Functions$1.class
_zem/io/opencensus/common/Functions$4.class
_zem/io/opencensus/common/ServerStatsEncoding.class
_zem/io/opencensus/common/AutoValue_Timestamp.class
_zem/io/opencensus/common/ServerStatsFieldEnums$Id.class
_zem/io/opencensus/common/ServerStatsFieldEnums$Size.class
_zem/io/opencensus/common/ServerStatsEncoding$1.class
_zem/io/opencensus/common/NonThrowingCloseable.class
_zem/io/opencensus/common/ExperimentalApi.class
_zem/io/opencensus/common/AutoValue_Duration.class
_zem/io/opencensus/common/ToLongFunction.class
_zem/io/opencensus/common/Functions$5.class
_zem/io/open

_zem/com/google/common/cache/LocalCache$SoftValueReference.class
_zem/com/google/common/cache/LocalCache$Strength$1.class
_zem/com/google/common/cache/LocalCache$Strength$2.class
_zem/com/google/common/cache/LocalCache$Strength$3.class
_zem/com/google/common/cache/LocalCache$Strength.class
_zem/com/google/common/cache/LocalCache$StrongAccessEntry.class
_zem/com/google/common/cache/LocalCache$StrongAccessWriteEntry.class
_zem/com/google/common/cache/LocalCache$StrongEntry.class
_zem/com/google/common/cache/LocalCache$StrongValueReference.class
_zem/com/google/common/cache/LocalCache$StrongWriteEntry.class
_zem/com/google/common/cache/LocalCache$ValueIterator.class
_zem/com/google/common/cache/LocalCache$ValueReference.class
_zem/com/google/common/cache/LocalCache$Values.class
_zem/com/google/common/cache/LocalCache$WeakAccessEntry.class
_zem/com/google/common/cache/LocalCache$WeakAccessWriteEntry.class
_zem/com/google/common/cache/LocalCache$WeakEntry.class
_zem/com/google/common/cache/

org/checkerframework/common/value/qual/BottomVal.class
org/checkerframework/common/value/qual/DoubleVal.class
org/checkerframework/common/value/qual/IntVal.class
org/checkerframework/common/value/qual/StaticallyExecutable.class
org/checkerframework/common/value/qual/StringVal.class
org/checkerframework/common/value/qual/UnknownVal.class
org/checkerframework/dataflow/
org/checkerframework/dataflow/qual/
org/checkerframework/dataflow/qual/Deterministic.class
org/checkerframework/dataflow/qual/Pure$Kind.class
org/checkerframework/dataflow/qual/Pure.class
org/checkerframework/dataflow/qual/SideEffectFree.class
org/checkerframework/dataflow/qual/TerminatesExecution.class
org/checkerframework/framework/
org/checkerframework/framework/qual/
org/checkerframework/framework/qual/AnnotatedFor.class
org/checkerframework/framework/qual/Bottom.class
org/checkerframework/framework/qual/ConditionalPostconditionAnnotation.class
org/checkerframework/framework/qual/DefaultFor.class
org/checkerframework/f

In [5]:
jar = r"C:/Users/Dilemre/Documents/GitHub/bitirme/zemberek-full.jar"
jvmpath = r"C:/Program Files/Java/jdk-17.0.2/bin/server/jvm.dll"
if not jpype.isJVMStarted():
    jpype.startJVM(jvmpath=jvmpath, classpath=jar)
    print("ok")

ok


In [15]:
try:
    TurkishTokenizer = JClass('zemberek.tokenization.TurkishTokenizer')
    Token = JClass('zemberek.tokenization.Token')
    TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
    TurkishSpellChecker = JClass('zemberek.normalization.TurkishSpellChecker')
    WordAnalysis =  JClass('zemberek.morphology.analysis.WordAnalysis')
    TurkishSentenceExtractor = JClass('zemberek.tokenization.TurkishSentenceExtractor')
except:
    print("wrong path")

In [16]:
morphology = TurkishMorphology.createWithDefaults()
tokenizer = TurkishTokenizer.ALL
spell_checker = TurkishSpellChecker(morphology)
extractor = TurkishSentenceExtractor.DEFAULT

In [8]:
import pandas as pd
import string

In [9]:
data = pd.read_csv("ndata.csv")

In [10]:
data

,info,title,caption,link,text
0,"19 Mart 2021, Cuma",MEHMET BARLAS,Türk siyasetinde Devlet Bahçeli bir tutarlılık...,https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nDün MHP'nin Büyük Kurultay'ında Genel Başk...
1,"18 Mart 2021, Perşembe",MEHMET BARLAS,Amerikan iç politikasında kimin kimi hedef ala...,https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nAmerikan iç politikasındaki gelişmeler kes...
2,"17 Mart 2021, Çarşamba",MEHMET BARLAS,"Amerikalılar, yalancı Amerikan başkanlarını ar...",https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nBugüne kadar her konuda Yunanistan'ın ve K...
3,"16 Mart 2021, Salı",MEHMET BARLAS,Darbeler geçmişte Türkiye’nin dış politikasını...,https://www.sabah.com.tr/yazarlar/barlas/2021/...,"\n\nMısır'la diplomatik ilişkilerin başlaması,..."
4,"15 Mart 2021, Pazartesi",MEHMET BARLAS,"Müflis Yunanistan, Türkiye’ye gövde gösterisi ...",https://www.sabah.com.tr/yazarlar/barlas/2021/...,\n\nTabii ki hiçbir ülkeyi küçük görmemek gere...
...,...,...,...,...,...
2410,"18 Şubat 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Takım ruhu lider yapar,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nAntalya'nın her alanda kurtuluşu takım ruh...
2411,"11 Şubat 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Önemli gün,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nBugün Antalya için çok önemli bir gün.\r\n...
2412,"28 Ocak 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Haydi maça,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nAntalya'da spor ve kardeşlik adına önemli ...
2413,"21 Ocak 2017, Cumartesi",ÖMER DURNA (AKDENİZ),Laf-ı güzaf,https://www.sabah.com.tr/yazarlar/bolgeler/ome...,\n\nDün bizim gazetenin manşet haberi Antalya ...


In [91]:
def dataCleaning(text):
    text = text.replace("\n", " ")
    text = text.lower()
    text = "".join([i for i in text if (i.isalnum() or i == " ")])
    return " ".join(text.split())

def wordTokenize(text):
    stopwords = [x.strip() for x in open('stop-words.txt','r', encoding="UTF8").read().split('\n')]
    text = text.split(" ")
    for word in text:
        if word in stopwords or word in string.whitespace:
            text.remove(word)
    return text

def sentTokenize(text):
    sent_list = []
    text = text.replace("\"", "")
    results: TurkishSentenceExtractor = extractor.fromDocument(jpype.JString(text))
    for result in results:
        result = dataCleaning(str(result))
        sent_list.append(result.split(","))
    return sent_list

def stems_lemmas(text):
    results: WordAnalysis = morphology.analyze(jpype.JString(text))
    stem, lemma = "-", "-"
    for result in results:
        stem = str(result.getStems()[0])
        lemma = str(result.getLemmas()[0])
    return [stem, lemma]

def wtLenDist(wt):
    wt_len = [len(word) for word in wt]
    wt_len_dist = dict()
    wt_len_dist.fromkeys(set(wt_len))
    for i in set(wt_len):
        wt_len_dist[i] = wt_len.count(i)
    return wt_len_dist

def stLenDist(st):
    st_len = [len(wordTokenize(sent[0])) for sent in st]
    st_len_dist = dict()
    st_len_dist.fromkeys(set(st_len))
    for i in set(st_len):
        st_len_dist[i] = st_len.count(i)
    return st_len_dist

In [92]:
stLenDist(sentTokenize(data.text[0]))

{1: 1,
 2: 3,
 3: 1,
 4: 2,
 5: 1,
 6: 3,
 9: 1,
 10: 2,
 11: 2,
 12: 3,
 13: 1,
 15: 2,
 18: 1}